# Benchmark Metrics Calculator

**Purpose:**
- Calculate Nash equilibrium and Monopoly prices for all experimental scenarios
- Support both symmetric and asymmetric firm configurations
- Verify the benchmark values specified in README.md

**Scenarios:**
1. 2-Firm Symmetric (c = 1, g = 2, μ = 0.40)
2. 3-Firm Symmetric (c = 1, g = 2, μ = 0.40)
3. 4-Firm Symmetric (c = 1, g = 2, μ = 0.40)
4. 3-Firm Asymmetric (c= [1.1, 1.00, 0.90], g = [2.08, 2.02, 1.96], μ = 0.30)



In [2]:
# =============================================================================
# N-Firm Logit Bertrand Pricing Model Numerical Solver
# Uses Newton's method to compute Nash equilibrium and monopoly prices
# =============================================================================

from __future__ import annotations
import numpy as np
from numpy.linalg import solve, LinAlgError
from dataclasses import dataclass
from typing import Callable, Tuple

def logit_shares(p: np.ndarray, g: np.ndarray, mu: float) -> np.ndarray:
    """
    Compute Logit demand shares
    
    Mathematical formula: s_i = exp((g_i - p_i)/μ) / [exp(0) + Σ_j exp((g_j - p_j)/μ)]
    
    Parameters:
        p: Price vector [p_0, p_1, ..., p_N]
        g: Product quality vector [g_0, g_1, ..., g_N] 
        mu: Product substitutability parameter)
    
    Returns:
        Market share vector [s_0, s_1, ..., s_N]
    """
    # Compute utility: u_i = (g_i - p_i) / μ
    util = (g - p) / mu
    
    # Numerical stability: prevent exp() overflow by shifting with max utility value
    m = max(0.0, float(util.max()))
    e = np.exp(util - m)  # Exponential utility for each product
    e0 = np.exp(-m)       # Outside option utility (u=0) also shifted by same amount
    
    # Compute denominator: outside option + sum of all product utilities
    denom = e0 + e.sum()
    
    # Return market share for each product
    return e / denom

def dsdpi_matrix(s: np.ndarray, mu: float) -> np.ndarray:
    """
    Compute Jacobian matrix of demand with respect to prices (∂s_i/∂p_j)
    
    Mathematical formulas:
        ∂s_i/∂p_i = -(1/μ) * s_i * (1 - s_i)  [diagonal elements]
        ∂s_i/∂p_j = (1/μ) * s_i * s_j         [off-diagonal elements]
    
    Parameters:
        s: Market share vector
        mu: Substitutability parameter
    
    Returns:
        N×N Jacobian matrix where M[i,j] = ∂s_i/∂p_j
    """
    N = len(s)
    M = np.empty((N, N), dtype=float)
    
    for i in range(N):
        for j in range(N):
            if i == j:
                # Diagonal: own price effect on own demand (negative effect)
                M[i, j] = -(1.0/mu) * s[i] * (1.0 - s[i])
            else:
                # Off-diagonal: competitor price effect on own demand (positive effect)
                M[i, j] = (1.0/mu) * s[i] * s[j]
    return M

def nash_F_and_J(p: np.ndarray, g: np.ndarray, c: np.ndarray, mu: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Compute Nash equilibrium first-order conditions F and Jacobian matrix J
    
    Nash equilibrium condition: each firm's first-order condition F_i = 0
    F_i = s_i + (p_i - c_i) * ∂s_i/∂p_i = 0
    
    Parameters:
        p: Price vector
        g: Quality vector  
        c: Cost vector
        mu: Substitutability parameter
    
    Returns:
        F: First-order condition vector [F_0, F_1, ..., F_N]
        J: Jacobian matrix (∂F_i/∂p_j)
    """
    # Compute market shares at current prices
    s = logit_shares(p, g, mu)
    
    # Compute Jacobian matrix of demand with respect to prices
    dsdpi = dsdpi_matrix(s, mu)
    N = len(p)
    
    # Build first-order conditions: F_i = s_i + (p_i - c_i) * ∂s_i/∂p_i
    F = s + (p - c) * np.diag(dsdpi)
    
    # Build Jacobian matrix: J[i,j] = ∂F_i/∂p_j
    J = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            # Compute second-order derivative terms
            if i == j:
                # ∂²s_i/∂p_i² = (1/μ²) * s_i * (1-s_i) * (1-2s_i)
                d2 = (1.0/mu**2) * s[i] * (1.0 - s[i]) * (1.0 - 2.0*s[i])
            else:
                # ∂²s_i/∂p_i∂p_j = -(1/μ²) * s_i * s_j * (1-2s_i)
                d2 = -(1.0/mu**2) * s[i] * s[j] * (1.0 - 2.0*s[i])
            
            # Jacobian matrix element: ∂F_i/∂p_j = ∂s_i/∂p_j + (p_i-c_i)*∂²s_i/∂p_i∂p_j + δ_ij*∂s_i/∂p_i
            J[i, j] = dsdpi[i, j] + (p[i] - c[i]) * d2 + (1 if i == j else 0) * dsdpi[i, i]
    
    return F, J

def monopoly_F_and_J(p: np.ndarray, g: np.ndarray, c: np.ndarray, mu: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Compute monopoly pricing first-order conditions F and Jacobian matrix J
    
    Monopoly condition: joint profit maximization with all prices changing simultaneously
    F_k = s_k + Σ_i (p_i - c_i) * ∂s_i/∂p_k = 0
    
    Parameters:
        p: Price vector
        g: Quality vector
        c: Cost vector  
        mu: Substitutability parameter
    
    Returns:
        F: First-order condition vector [F_0, F_1, ..., F_N]
        J: Jacobian matrix (∂F_k/∂p_j)
    """
    # Compute market shares at current prices
    s = logit_shares(p, g, mu)
    
    # Compute Jacobian matrix of demand with respect to prices
    dsdpi = dsdpi_matrix(s, mu)
    N = len(p)
    
    # Build first-order conditions: F_k = s_k + Σ_i (p_i - c_i) * ∂s_i/∂p_k
    F = s + dsdpi.T @ (p - c)
    
    # Build Jacobian matrix: J[k,j] = ∂F_k/∂p_j
    J = np.zeros((N, N))
    for k in range(N):
        for j in range(N):
            # ∂s_k/∂p_j term
            ds_k_dpj = dsdpi[k, j]
            
            # Σ_i (p_i - c_i) * ∂²s_i/∂p_k∂p_j term
            term = 0.0
            for i in range(N):
                if i == k:
                    # Second-order derivative when i = k
                    if j == i:
                        d2 = (1.0/mu**2) * s[i] * (1.0 - s[i]) * (1.0 - 2.0*s[i])
                    else:
                        d2 = -(1.0/mu**2) * s[i] * s[j] * (1.0 - 2.0*s[i])
                else:
                    # Second-order derivative when i ≠ k
                    d2 = (1.0/mu) * (dsdpi[i, j] * s[k] + s[i] * dsdpi[k, j])
                
                # Accumulate terms
                term += (1 if i == j else 0) * dsdpi[i, k] + (p[i] - c[i]) * d2
            
            # Jacobian matrix element
            J[k, j] = ds_k_dpj + term
    
    return F, J

@dataclass
class NewtonResult:
    """
    Newton's method solution result
    
    Attributes:
        p: Solved price vector
        converged: Whether convergence was achieved
        iters: Number of iterations
    """
    p: np.ndarray
    converged: bool
    iters: int

def newton_vec(FJ, p0: np.ndarray, g: np.ndarray, c: np.ndarray, mu: float,
               tol: float = 1e-12, max_iter: int = 200, damping: float = 1.0) -> NewtonResult:
    """
    Vectorized Newton-Raphson method for solving nonlinear equation systems
    
    Algorithm: p_{k+1} = p_k - damping * J^{-1} * F(p_k)
    
    Parameters:
        FJ: Function returning (F, J) i.e., first-order conditions and Jacobian matrix
        p0: Initial price vector
        g: Quality vector
        c: Cost vector
        mu: Substitutability parameter
        tol: Convergence tolerance
        max_iter: Maximum number of iterations
        damping: Damping factor (prevents divergence)
    
    Returns:
        NewtonResult: Solution result
    """
    p = np.array(p0, dtype=float).copy()
    
    for t in range(1, max_iter + 1):
        # Compute first-order conditions and Jacobian matrix at current point
        F, J = FJ(p, g, c, mu)
        
        try:
            # Solve linear system J * step = F
            step = solve(J, F)
        except LinAlgError:
            # If Jacobian matrix is singular, use pseudo-inverse
            step = np.linalg.pinv(J) @ F
        
        # Update prices: p_{k+1} = p_k - damping * step
        p_next = p - damping * step
        
        # Check convergence: ||p_{k+1} - p_k||_∞ < tol
        if np.linalg.norm(p_next - p, ord=np.inf) < tol:
            return NewtonResult(p=p_next, converged=True, iters=t)
        
        p = p_next
    
    # Reached maximum iterations without convergence
    return NewtonResult(p=p, converged=False, iters=max_iter)

def solve_nash(g: np.ndarray, c: np.ndarray, mu: float, p0: np.ndarray | None = None, **kwargs) -> NewtonResult:
    """
    Solve for Nash equilibrium prices
    
    Parameters:
        g: Quality vector
        c: Cost vector
        mu: Substitutability parameter
        p0: Initial prices (default: c + 0.6)
        **kwargs: Other Newton method parameters
    
    Returns:
        NewtonResult: Nash equilibrium solution result
    """
    if p0 is None:
        # Smart initial value: cost plus reasonable profit margin
        p0 = c + 0.6
    return newton_vec(nash_F_and_J, p0, g, c, mu, **kwargs)

def solve_monopoly(g: np.ndarray, c: np.ndarray, mu: float, p0: np.ndarray | None = None, **kwargs) -> NewtonResult:
    """
    Solve for monopoly prices
    
    Parameters:
        g: Quality vector
        c: Cost vector
        mu: Substitutability parameter
        p0: Initial prices (default: c + 0.9)
        **kwargs: Other Newton method parameters
    
    Returns:
        NewtonResult: Monopoly pricing solution result
    """
    if p0 is None:
        # Smart initial value: cost plus higher profit margin (monopoly prices are typically higher)
        p0 = c + 0.9
    return newton_vec(monopoly_F_and_J, p0, g, c, mu, **kwargs)

def shares_profits(p: np.ndarray, g: np.ndarray, c: np.ndarray, mu: float):
    """
    Compute market shares and profits at given prices
    
    Parameters:
        p: Price vector
        g: Quality vector
        c: Cost vector
        mu: Substitutability parameter
    
    Returns:
        s: Market share vector
        prof: Profit vector (π_i = (p_i - c_i) * s_i)
    """
    s = logit_shares(p, g, mu)
    prof = (p - c) * s
    return s, prof

def summarize(title: str, p: np.ndarray, g: np.ndarray, c: np.ndarray, mu: float):
    """
    Print summary information of pricing results
    
    Parameters:
        title: Title
        p: Price vector
        g: Quality vector
        c: Cost vector
        mu: Substitutability parameter
    """
    s, prof = shares_profits(p, g, c, mu)
    
    print(f"=== {title} ===")
    for i in range(len(p)):
        print(f"firm {i}: price={p[i]:.6f}, share={s[i]:.6f}, profit={prof[i]:.6f}, "
              f"cost={c[i]:.6f}, quality={g[i]:.6f}")
    
    print(f"TOTAL: share={s.sum():.6f}, profit={prof.sum():.6f}")


In [6]:
# =============================================================================
# Example 1: Symmetric Two-Firm Scenario
# =============================================================================

if __name__ == "__main__":

    mu = 0.40
    g = np.array([2.00, 2.00])
    c = np.array([1.00, 1.00])

    resN = solve_nash(g, c, mu)
    summarize("Nash (Bertrand)", resN.p, g, c, mu)
    print(f"Nash converged: {resN.converged} in {resN.iters} iterations.\n")

    resM = solve_monopoly(g, c, mu)
    summarize("Monopoly (Joint Profit)", resM.p, g, c, mu)
    print(f"Monopoly converged: {resM.converged} in {resM.iters} iterations.")

    pr_low = resN.p - 0.1*(resM.p-resN.p)
    pr_high = resM.p + 0.1*(resM.p-resN.p)
    print(f'price range: [{pr_low[0]:.6f}, {pr_high[0]:.6f}]')

=== Nash (Bertrand) ===
firm 0: price=1.676696, share=0.408892, profit=0.276696, cost=1.000000, quality=2.000000
firm 1: price=1.676696, share=0.408892, profit=0.276696, cost=1.000000, quality=2.000000
TOTAL: share=0.817784, profit=0.553391
Nash converged: True in 4 iterations.

=== Monopoly (Joint Profit) ===
firm 0: price=2.070585, share=0.313186, profit=0.335292, cost=1.000000, quality=2.000000
firm 1: price=2.070585, share=0.313186, profit=0.335292, cost=1.000000, quality=2.000000
TOTAL: share=0.626372, profit=0.670585
Monopoly converged: True in 5 iterations.
price range: [1.637307, 2.109974]


In [8]:
# =============================================================================
# Example 2: Symmetric Three-Firm Scenario
# =============================================================================

if __name__ == "__main__":
    # Example with 3 symmetric firms
    mu = 0.40
    g = np.array([2.00, 2.00, 2.00])
    c = np.array([1.00, 1.00, 1.00])

    resN = solve_nash(g, c, mu)
    summarize("Nash (Bertrand)", resN.p, g, c, mu)
    print(f"Nash converged: {resN.converged} in {resN.iters} iterations.\n")

    resM = solve_monopoly(g, c, mu)
    summarize("Monopoly (Joint Profit)", resM.p, g, c, mu)
    print(f"Monopoly converged: {resM.converged} in {resM.iters} iterations.")

    pr_low = resN.p - 0.1*(resM.p-resN.p)
    pr_high = resM.p + 0.1*(resM.p-resN.p)
    print(f'price range: [{pr_low[0]:.6f}, {pr_high[0]:.6f}]')

=== Nash (Bertrand) ===
firm 0: price=1.570796, share=0.299224, profit=0.170796, cost=1.000000, quality=2.000000
firm 1: price=1.570796, share=0.299224, profit=0.170796, cost=1.000000, quality=2.000000
firm 2: price=1.570796, share=0.299224, profit=0.170796, cost=1.000000, quality=2.000000
TOTAL: share=0.897672, profit=0.512388
Nash converged: True in 4 iterations.

=== Monopoly (Joint Profit) ===
firm 0: price=2.174924, share=0.219851, profit=0.258308, cost=1.000000, quality=2.000000
firm 1: price=2.174924, share=0.219851, profit=0.258308, cost=1.000000, quality=2.000000
firm 2: price=2.174924, share=0.219851, profit=0.258308, cost=1.000000, quality=2.000000
TOTAL: share=0.659553, profit=0.774924
Monopoly converged: True in 5 iterations.
price range: [1.510383, 2.235337]


In [23]:
# =============================================================================
# Example 3: Symmetric Four-Firm Scenario
# =============================================================================

if __name__ == "__main__":

    mu = 0.40
    g = np.array([2.00, 2.00, 2.00, 2.00])
    c = np.array([1.00, 1.00, 1.00, 1.00])

    resN = solve_nash(g, c, mu)
    summarize("Nash (Bertrand)", resN.p, g, c, mu)
    print(f"Nash converged: {resN.converged} in {resN.iters} iterations.\n")

    resM = solve_monopoly(g, c, mu)
    summarize("Monopoly (Joint Profit)", resM.p, g, c, mu)
    print(f"Monopoly converged: {resM.converged} in {resM.iters} iterations.")

    pr_low = resN.p - 0.1*(resM.p-resN.p)
    pr_high = resM.p + 0.1*(resM.p-resN.p)
    print(f'price range: [{pr_low[0]:.6f}, {pr_high[0]:.6f}]')

=== Nash (Bertrand) ===
firm 0: price=1.521137, share=0.232447, profit=0.121137, cost=1.000000, quality=2.000000
firm 1: price=1.521137, share=0.232447, profit=0.121137, cost=1.000000, quality=2.000000
firm 2: price=1.521137, share=0.232447, profit=0.121137, cost=1.000000, quality=2.000000
firm 3: price=1.521137, share=0.232447, profit=0.121137, cost=1.000000, quality=2.000000
TOTAL: share=0.929789, profit=0.484547
Nash converged: True in 4 iterations.

=== Monopoly (Joint Profit) ===
firm 0: price=2.252047, share=0.170131, profit=0.213012, cost=1.000000, quality=2.000000
firm 1: price=2.252047, share=0.170131, profit=0.213012, cost=1.000000, quality=2.000000
firm 2: price=2.252047, share=0.170131, profit=0.213012, cost=1.000000, quality=2.000000
firm 3: price=2.252047, share=0.170131, profit=0.213012, cost=1.000000, quality=2.000000
TOTAL: share=0.680523, profit=0.852047
Monopoly converged: True in 5 iterations.
price range: [1.448046, 2.325138]


In [22]:
# =============================================================================
# Example 4: Asymmetric Three-Firm Scenario
# =============================================================================

if __name__ == "__main__":

    mu = 0.40
    g = np.array([2.08, 2.02, 1.96])
    c = np.array([1.10, 1.00, 0.95])

    resN = solve_nash(g, c, mu)
    summarize("Nash (Bertrand)", resN.p, g, c, mu)
    print(f"Nash converged: {resN.converged} in {resN.iters} iterations.\n")

    resM = solve_monopoly(g, c, mu)
    summarize("Monopoly (Joint Profit)", resM.p, g, c, mu)
    print(f"Monopoly converged: {resM.converged} in {resM.iters} iterations.")

=== Nash (Bertrand) ===
firm 0: price=1.662311, share=0.288650, profit=0.162311, cost=1.100000, quality=2.080000
firm 1: price=1.577379, share=0.307214, profit=0.177379, cost=1.000000, quality=2.020000
firm 2: price=1.523510, share=0.302541, profit=0.173510, cost=0.950000, quality=1.960000
TOTAL: share=0.898405, profit=0.513201
Nash converged: True in 4 iterations.

=== Monopoly (Joint Profit) ===
firm 0: price=2.277361, share=0.207429, profit=0.244218, cost=1.100000, quality=2.080000
firm 1: price=2.177361, share=0.229244, profit=0.269903, cost=1.000000, quality=2.020000
firm 2: price=2.127361, share=0.223584, profit=0.263239, cost=0.950000, quality=1.960000
TOTAL: share=0.660257, profit=0.777361
Monopoly converged: True in 5 iterations.


In [21]:
# =============================================================================
# Example 5: Asymmetric Four-Firm Scenario
# =============================================================================

if __name__ == "__main__":

    mu = 0.40
    g = np.array([2.08, 2.02, 1.96, 1.90])
    c = np.array([1.10, 1.00, 0.95, 0.90])

    resN = solve_nash(g, c, mu)
    summarize("Nash (Bertrand)", resN.p, g, c, mu)
    print(f"Nash converged: {resN.converged} in {resN.iters} iterations.\n")

    resM = solve_monopoly(g, c, mu)
    summarize("Monopoly (Joint Profit)", resM.p, g, c, mu)
    print(f"Monopoly converged: {resM.converged} in {resM.iters} iterations.")

=== Nash (Bertrand) ===
firm 0: price=1.614955, share=0.223233, profit=0.114955, cost=1.100000, quality=2.080000
firm 1: price=1.526219, share=0.239861, profit=0.126219, cost=1.000000, quality=2.020000
firm 2: price=1.473316, share=0.235643, profit=0.123316, cost=0.950000, quality=1.960000
firm 3: price=1.420471, share=0.231465, profit=0.120471, cost=0.900000, quality=1.900000
TOTAL: share=0.930202, profit=0.484961
Nash converged: True in 5 iterations.

=== Monopoly (Joint Profit) ===
firm 0: price=2.353934, share=0.160829, profit=0.201669, cost=1.100000, quality=2.080000
firm 1: price=2.253934, share=0.177744, profit=0.222879, cost=1.000000, quality=2.020000
firm 2: price=2.203934, share=0.173355, profit=0.217376, cost=0.950000, quality=1.960000
firm 3: price=2.153934, share=0.169075, profit=0.212009, cost=0.900000, quality=1.900000
TOTAL: share=0.681004, profit=0.853934
Monopoly converged: True in 5 iterations.
